In [11]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [12]:
def ADCSBM(n,k):
    A = np.zeros((n,n))
    theta = np.random.beta(1, 4, n)
    theta2 = theta/sum(theta)
    B=np.array([[0.9,0.1,0.1],[0.1,0.6,0.1],[0.1,0.1,0.3]])
    for i in range(0,n):
        for j in range(i+1,n):
            if y[i] == y[j] == 1:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[0,0])
            elif y[i] == y[j] == 2:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[1,1])
            elif y[i] == y[j] == 3:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[2,2])
            else:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*0.1)
    for i in range(0,n):
        for j in range(0,i):
            A[i,j]=A[j,i]
    return A

In [13]:
def gety(n):
    y = np.zeros((n,1))
    #np.random.seed(2)
    for i in range(0,n):
        y[i,0] = np.random.choice(np.arange(1, 4), p=[0.2, 0.3,0.5])
    return(y)

In [14]:
def EncoderLDA(A):
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
#ASE*LDA
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    newx=([])
    d=3
    newx=np.concatenate(U[:,0:d]*s[0:d]**0.5)
    newx=np.reshape(newx,(n,d))
    clf2 = LinearDiscriminantAnalysis()
    #print(np.shape(newx))
    newx_train, newx_test =newx[train_index], newx[test_index]
    clf2.fit(newx_train, y_train.ravel())
    ASELDAACC=clf2.score(newx_test, y_test.ravel())
    #time.sleep(2) 
    end1 = time.time()
    ASELDATIME=end1-begin1
    eigenvalues=s**2
    Top=eigenvalues[:10]
#nk,w,Z
    begin5 = time.time()
    global k
    nk = np.zeros((1,k))
    for i in range(0,len(y_train)-1):
        nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
    w = np.zeros((int(np.size(y_train)),k))
    for i in range(0,int(np.size(y_train))):
        k=int(y_train[i])
        w[i][k-1]=1/nk[0,k-1]
    z_train=np.matmul(X_train,w)
    z_test = np.matmul(X_test,w)
    #time.sleep(5)
    end5 = time.time()
    zTIME=end5 - begin5
#AEE*KNN
    begin2 = time.time()
    neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
    neigh.fit(z_train, y_train.ravel())
    AEEKNNACC = neigh.score(z_test, y_test)
    #time.sleep(3) 
    end2 = time.time()
    AEEKNNTIME=end2-begin2
#AEE*LDA
    begin3 = time.time()
    clf4 = LinearDiscriminantAnalysis()
    clf4.fit(z_train,y_train.ravel())
    AEELDAACC = clf4.score(z_test, y_test.ravel())
    #time.sleep(4) 
    end3 = time.time()
    AEELDATIME=end3-begin3
#Get result
    Acc=np.array([ASELDAACC,AEEKNNACC,AEELDAACC])
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    print(f"When n = {n}")
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")

In [15]:
for n in range (100,3100,100):
    k=3
    y=gety(n)
    A=ADCSBM(n,k=k)
    EncoderLDA(A)

When n = 100
Accuracy for each method: [0.7  0.45 0.7 ]
Time for each method: [0.01320863 0.0137167  0.00251412]
When n = 200
Accuracy for each method: [0.625 0.625 0.8  ]
Time for each method: [0.01898599 0.00471401 0.00469089]
When n = 300
Accuracy for each method: [0.56666667 0.63333333 0.75      ]
Time for each method: [0.02113605 0.0048182  0.00239992]
When n = 400
Accuracy for each method: [0.55  0.725 0.7  ]
Time for each method: [0.04724813 0.00701475 0.00372982]
When n = 500
Accuracy for each method: [0.65 0.77 0.76]
Time for each method: [0.17776585 0.01954985 0.00445724]
When n = 600
Accuracy for each method: [0.65       0.71666667 0.8       ]
Time for each method: [0.16035485 0.01332593 0.00494313]
When n = 700
Accuracy for each method: [0.62142857 0.82857143 0.78571429]
Time for each method: [0.22425294 0.01314211 0.0070703 ]
When n = 800
Accuracy for each method: [0.74375 0.7375  0.79375]
Time for each method: [0.34187603 0.01285529 0.00547123]
When n = 900
Accuracy for e